# Get your API KEY

In [1]:
import os
os.environ["GROQ_API_KEY"]="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"

In [2]:
!pip install -q langchain_groq langchain-core requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 3.9 MB/s eta 0:00:00


In [1]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

True

# Tool Creation

In [2]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [3]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [4]:
multiply.name

'multiply'

In [5]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [6]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

# Tool Binding

In [7]:
model = ChatGroq(model="llama-3.1-8b-instant")

In [8]:
model.invoke('hi')

AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.016412583, 'prompt_time': 0.002017675, 'queue_time': 0.049922275, 'total_time': 0.018430258}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_8804b970d6', 'finish_reason': 'stop', 'logprobs': None}, id='run--83298842-6bb4-40c2-9239-f6b970a670c7-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})

In [9]:
llm_with_tools = model.bind_tools([multiply])

In [10]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm functioning properly, thank you for asking. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 234, 'total_tokens': 252, 'completion_time': 0.027666855, 'prompt_time': 0.014153367, 'queue_time': 0.049935503, 'total_time': 0.041820222}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_28116a6b5f', 'finish_reason': 'stop', 'logprobs': None}, id='run--13fc3688-00d4-4688-ad19-959a3e5fe43d-0', usage_metadata={'input_tokens': 234, 'output_tokens': 18, 'total_tokens': 252})

In [11]:
query = HumanMessage('can you multiply 3 with 1000')

In [12]:
messages = [query]

In [13]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [14]:
result = llm_with_tools.invoke(messages)

In [15]:
messages.append(result)

In [19]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_p09c', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 239, 'total_tokens': 259, 'completion_time': 0.03600864, 'prompt_time': 0.01738249, 'queue_time': 0.215575177, 'total_time': 0.05339113}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_55d70a61e4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7c74ad6e-6cff-4e95-b9fb-c487d6713f51-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_p09c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 239, 'output_tokens': 20, 'total_tokens': 259})]

In [16]:
tool_result = multiply.invoke(result.tool_calls[0])

In [17]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='h55sv918h')

In [18]:
messages.append(tool_result)

In [19]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'h55sv918h', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 239, 'total_tokens': 259, 'completion_time': 0.032248113, 'prompt_time': 0.014391743, 'queue_time': 0.049762847, 'total_time': 0.046639856}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_28116a6b5f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7f58e3ce-29d5-4c52-889e-6269135f9b29-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'h55sv918h', 'type': 'tool_call'}], usage_metadata={'input_tokens': 239, 'output_tokens': 20, 'total_tokens': 259}),
 ToolMessage(content='3000', name='multiply', tool_call_id='h55sv918h')]

In [21]:
llm_with_tools.invoke(messages).content

'The result of multiplying 3 with 1000 is 3000.'

# Example of Realtime Currecy Converter

In [25]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/2e2ebcfbc9dcb0d62da963e4/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [26]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [27]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1749254401,
 'time_last_update_utc': 'Sat, 07 Jun 2025 00:00:01 +0000',
 'time_next_update_unix': 1749340801,
 'time_next_update_utc': 'Sun, 08 Jun 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.817}

In [28]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [29]:
# tool binding
llm = ChatGroq(model="llama-3.1-8b-instant")

In [30]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [37]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that value can you convert 10 inr to usd using that conversion rate')]

In [38]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that value can you convert 10 inr to usd using that conversion rate', additional_kwargs={}, response_metadata={})]

In [39]:
ai_message = llm_with_tools.invoke(messages)

In [40]:
messages.append(ai_message)

In [41]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_2xpz',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10, 'conversion_factor': 0.0137},
  'id': 'call_r4tw',
  'type': 'tool_call'}]

In [42]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [43]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that value can you convert 10 inr to usd using that conversion rate', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2xpz', 'function': {'arguments': '{"base_currency":"INR","target_currency":"USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_r4tw', 'function': {'arguments': '{"base_currency_value":10,"conversion_factor":0.0137}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 709, 'total_tokens': 757, 'completion_time': 0.064, 'prompt_time': 0.045551761, 'queue_time': -0.214619244, 'total_time': 0.109551761}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--57f35cbf-76ea-4642-8fe7-f4dcc526f602-0', tool_calls=[{'name': 'get_conversion_factor', 'args':

In [44]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 0.01165, and 10 INR is equivalent to 0.1165 USD.'

In [45]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

<ipython-input-45-23d1b63d4a11>:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [22]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})

NameError: name 'agent_executor' is not defined